**Import libraries that will be used.**

In [1]:
import numpy as np
import pandas as pd
import pickle
import re
from autocorrect import Speller
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 2500)
pd.set_option('display.max_colwidth', 100)

**Load the pickle file into a dataframe.**

In [2]:
rec = pd.read_pickle('rec.pkl')
rec.head()

,product id,company,order id,order total,quantity,price,cost,product name
0,2310.0,Company0,3758,64.29,1,57.20,44.00,"Basic Rollators, Green 1Each/Each"
1,177.0,Company1,23,29.99,4,5.00,1.87,"Urinary Drain Bags,0.000 1Each/Each"
2,1.0,Company1,9531,78.73,1,68.78,52.50,"SensiCare Nitrile Exam Gloves, Blue, XX-Large 800Each/Case"
3,983.0,None,29,29.55,1,19.56,12.62,Basket for 2-Button Walkers 1Each/Each
4,991.0,Company3,30,46.00,1,36.05,25.75,Tens Units 1Each/Each


**Save each unique product ID and the first product name into a separate dataframe.**

In [3]:
corpus = rec.groupby('product id')['product name'].agg({'product name': 'first'})
corpus.head(20)

,product name
product id,
1.0,"SensiCare Nitrile Exam Gloves, Blue, XX-Large 800Each/Case"
11.0,"SensiCare 200 Nitrile Exam Gloves, Blue, Large"
14.0,"Asorbs Ultra-Soft Plus Briefs, Large"
15.0,"Asorbs Ultra-Soft Plus Briefs, Medium"
19.0,Disposable Forehead Thermometer 100Each/Box
20.0,"BariBrief Bariatric Briefs, Bariatric 32Each/Case"
22.0,"BariBrief Bariatric Briefs, Bariatric 8Each/Bag"
26.0,Capri Plus Bladder Control Pads
28.0,Capri Plus Bladder Control Pads


**Create a dictionary to make words spelt differently more uniform.**

In [4]:
converter = {'bx': 'box',
             'yrds': 'yard',
             'yd': 'yard',
             'oz': 'ounce',
             'lb': 'pound',
             'lbs': 'pound',
             'w/': 'with ',
             'x-large': 'xl'}

**Create a function to clean text in column product name.**

In [5]:
def clean_text(text):
    text = text.lower() #makes all letters lowercase
    text = re.sub('[^\x00-\x7F]+', ' ', text) #remove non-ASCII characters
    text = re.sub('\s[-]', ' ', text) #remove hyphen if whitespace before hyphen
    text = re.sub('["&\'\(\)]', '', text) #remove double and single quotes and '&' character
    text = re.sub('\[\w*\]','', text) #remove brackets and any alphanumeric characters within
    text = re.sub('[,]', ' ', text) # remove commas and replace with a space
    
    for key,val in converter.items():
        if key in text:
            text = text.replace(key,val)
    
    spell = Speller()
    text = spell(text)
    
    wnl = WordNetLemmatizer()
    clean_verb = ''
    clean_text = ''
    
    word_list1 = word_tokenize(text) # tokenizes text
    for word in word_list1:
        clean_verb += wnl.lemmatize(word, pos = 'v') + ' '
    
    word_list2 = word_tokenize(clean_verb)
    for word in word_list2:
        clean_text += wnl.lemmatize(word, pos = 'n') + ' '
        
    words_to_keep = [word for word in clean_text.split() if len(word) > 1]
    final_text = ' '.join(words_to_keep)
    
    return final_text

In [6]:
corpus['clean name'] = corpus['product name'].apply(clean_text)

**Remove certain words from product name.**

In [7]:
add_stopwords = ['ith', 'nth']
stop_set = set(stopwords.words('english') + add_stopwords)

In [8]:
def remove_stopwords(text):
    final_text = ''
    list_of_words = word_tokenize(text)
    imp_words = [word for word in list_of_words if word not in stop_set]
    for word in imp_words:
        final_text += word + ' '
    
    return final_text

In [9]:
corpus['final name'] = corpus['clean name'].apply(remove_stopwords)
corpus[['final name']].head(20)

,final name
product id,
1.0,sensical nitrile exam glove blue xxx 800each/case
11.0,sensical 200 nitrile exam glove blue large
14.0,sorb ultra-soft plus brief large
15.0,sorb ultra-soft plus brief medium
19.0,disposable forehead thermometer 100each/box
20.0,baribrief bariatric brief bariatric 32each/case
22.0,baribrief bariatric brief bariatric 8each/bag
26.0,capri plus bladder control pad
28.0,capri plus bladder control pad


**Create a TF-IDF Vectorizer.**

In [10]:
vect = TfidfVectorizer(token_pattern = '[a-zA-Z0-9.\-/]+')
vectorize = vect.fit_transform(corpus['final name'])

tf = pd.DataFrame(vectorize.toarray(), columns = vect.get_feature_names(), index = corpus.index)
tf.head()

0.0  0.00  0.000  0.130  0.150  0.17  0.250  0.4  0.5  0.50  \
product id                                                                
1.0         0.0   0.0    0.0    0.0    0.0   0.0    0.0  0.0  0.0   0.0   
11.0        0.0   0.0    0.0    0.0    0.0   0.0    0.0  0.0  0.0   0.0   
14.0        0.0   0.0    0.0    0.0    0.0   0.0    0.0  0.0  0.0   0.0   
15.0        0.0   0.0    0.0    0.0    0.0   0.0    0.0  0.0  0.0   0.0   
19.0        0.0   0.0    0.0    0.0    0.0   0.0    0.0  0.0  0.0   0.0   

            0.500  0.6  0.750  0.850  1-3/8  1-channel  1-ply  1.00  1.000  \
product id                                                                   
1.0           0.0  0.0    0.0    0.0    0.0        0.0    0.0   0.0    0.0   
11.0          0.0  0.0    0.0    0.0    0.0        0.0    0.0   0.0    0.0   
14.0          0.0  0.0    0.0    0.0    0.0        0.0    0.0   0.0    0.0   
15.0          0.0  0.0    0.0    0.0    0.0        0.0    0.0   0.0    0.0   
19.0          0.0  0.0    0.0    0.0    0.0        0.0    0.0   0.0    0.0   

            1.1cc  1.5  1.5cc  1.5x1.5  1.7  1.75  1/2   10  10.000  10.5  \
product id                                                                  
1.0           0.0  0.0    0.0      0.0  0.0   0.0  0.0  0.0     0.0   0.0   
11.0          0.0  0.0    0.0      0.0  0.0   0.0  0.0  0.0     0.0   0.0   
14.0          0.0  0.0    0.0      0.0  0.0   0.0  0.0  0.0     0.0   0.0   
15.0          0.0  0.0    0.0      0.0  0.0   0.0  0.0  0.0     0.0   0.0   
19.0          0.0  0.0    0.0      0.0  0.0   0.0  0.0  0.0     0.0   0.0   

            10.75  10/100/cs  10/pkg  100  100/cs  1000  1000.00  \
product id                                                         
1.0           0.0        0.0     0.0  0.0     0.0   0.0      0.0   
11.0          0.0        0.0     0.0  0.0     0.0   0.0      0.0   
14.0          0.0        0.0     0.0  0.0     0.0   0.0      0.0   
15.0          0.0        0.0     0.0  0.0     0.0   0.0      0.0   
19.0          0.0        0.0     0.0  0.0     0.0   0.0      0.0   

            1000each/box  1000each/case  100each/box  100each/case  \
product id                                                           
1.0                  0.0            0.0      0.00000           0.0   
11.0                 0.0            0.0      0.00000           0.0   
14.0                 0.0            0.0      0.00000           0.0   
15.0                 0.0            0.0      0.00000           0.0   
19.0                 0.0            0.0      0.42254           0.0   

            100each/pack  10215-s  105  1080each/case  10each/bag  10each/box  \
product id                                                                      
1.0                  0.0      0.0  0.0            0.0         0.0         0.0   
11.0                 0.0      0.0  0.0            0.0         0.0         0.0   
14.0                 0.0      0.0  0.0            0.0         0.0         0.0   
15.0                 0.0      0.0  0.0            0.0         0.0         0.0   
19.0                 0.0      0.0  0.0            0.0         0.0         0.0   

            10each/case  10each/tray  10fr  10x13  10x14   11  11.5  1108  \
product id                                                                  
1.0                 0.0          0.0   0.0    0.0    0.0  0.0   0.0   0.0   
11.0                0.0          0.0   0.0    0.0    0.0  0.0   0.0   0.0   
14.0                0.0          0.0   0.0    0.0    0.0  0.0   0.0   0.0   
15.0                0.0          0.0   0.0    0.0    0.0  0.0   0.0   0.0   
19.0                0.0          0.0   0.0    0.0    0.0  0.0   0.0   0.0   

            113.00  118.0  118.00  118.28  118.29  11838-1   12  12.5  \
product id                                                              
1.0            0.0    0.0     0.0     0.0     0.0      0.0  0.0   0.0   
11.0           0.0    0.0     0.0     0.0     0.0      0.0  0.0   0.0   
14.0           0.0    0.0     0

**Apply cosine similarity to the product name and TF-IDF dataframe to find similary products.**

In [11]:
#returns up to top 10 similar products
def recommender(ID, tfidf):
    q_vector = tfidf.loc[[ID],:]
    similarities = cosine_similarity(q_vector, tfidf)
    df = pd.DataFrame(data = similarities).T
    df.set_index([corpus.index, corpus['product name']], inplace = True)
    df.rename(columns = {df.columns[0]: 'Similarity Score'}, inplace = True)
            
    sim_df = df[df['Similarity Score'] > 0.20] #return only products that have cosine similarity greater than 0.20
    sim_df.sort_values(by = 'Similarity Score', ascending = False, inplace = True)
    
    return sim_df.iloc[1:11,:]

In [12]:
recommender(1.0, tf)

,,Similarity Score
product id,product name,
918.0,"SensiCare Nitrile Exam Gloves, Blue, Medium 1500Each/Case",0.561284
921.0,"SensiCare Nitrile Exam Gloves, Blue, Large 1500Each/Case",0.558910
915.0,"SensiCare Nitrile Exam Gloves, Blue, Small 1500Each/Case",0.548331
11.0,"SensiCare 200 Nitrile Exam Gloves, Blue, Large",0.533939
802.0,"SensiCare Silk Nitrile Exam Gloves, Dark Blue, Large",0.502313
799.0,"SensiCare Silk Nitrile Exam Gloves, Dark Blue, Small",0.494591
4523.0,"SensiCare Ice Blue Nitrile Exam Gloves, Violet Blue, Medium 2500Each/Case",0.461511
784.0,"SensiCare Ice Blue Nitrile Exam Gloves, Violet Blue, Medium 2500Each/Case",0.461511
785.0,"SensiCare Ice Blue Nitrile Exam Gloves, Violet Blue, Large 2500Each/Case",0.460444
